## Introduction

Is is possible to predict the gender of a character in a story using the pronouns around them?

Yes

He and she are objective pronouns

Him and her are subjective pronouns

His and her are possessive adjectives

His and hers are possessive pronouns

"Her" is possessive when followed by a noun.

## Methodology

## Hypothesis



pip installs, library imports, and global vars

In [ ]:
# %pip install sparknlp
# %pip install pyspark
# %pip install spacy

In [ ]:
import sparknlp
import os
import spacy
import pyspark
from pyspark.sql import SparkSession
from tqdm import tqdm
import datetime
import pandas as pd
from sparknlp.pretrained import PretrainedPipeline

In [24]:
spark = sparknlp.start()
root = os.path.dirname(os.path.realpath('cormac.ipynb'))
pipeline = PretrainedPipeline("explain_document_ml")

explain_document_ml download started this may take some time.
Approx size to download 9 MB
[OK!]


get corpus

In [ ]:
book_list = ['pg1342.txt', 'pg13600.txt', 'pg19699.txt', 'pg19846.txt', 'pg27478.txt', 'pg27479.txt', 'pg27480.txt', 'pg30073.txt', 'pg30685.txt', 'pg30935.txt', 'pg30976.txt', 'pg31156.txt', 'pg31329.txt', 'pg31447.txt', 'pg31641.txt', 'pg31793.txt', 'pg31855.txt', 'pg31950.txt', 'pg32063.txt', 'pg32097.txt', 'pg32182.txt', 'pg32294.txt', 'pg32423.txt', 'pg32607.txt', 'pg32689.txt', 'pg32758.txt', 'pg32783.txt', 'pg32860.txt', 'pg32940.txt', 'pg32975.txt', 'pg33052.txt', 'pg33127.txt', 'pg33189.txt', 'pg33239.txt', 'pg33295.txt', 'pg33365.txt', 'pg33427.txt', 'pg33477.txt', 'pg33550.txt', 'pg33614.txt', 'pg33698.txt', 'pg33750.txt', 'pg33991.txt', 'pg34018.txt', 'pg34047.txt', 'pg34082.txt', 'pg34116.txt', 'pg34162.txt', 'pg34209.txt', 'pg34312.txt', 'pg34405.txt', 'pg34533.txt', 'pg34612.txt', 'pg34702.txt', 'pg34751.txt', 'pg34878.txt', 'pg34992.txt', 'pg35092.txt', 'pg35169.txt', 'pg35236.txt', 'pg35306.txt', 'pg35398.txt', 'pg35473.txt', 'pg35561.txt', 'pg35606.txt', 'pg35747.txt', 'pg35925.txt', 'pg36104.txt', 'pg36226.txt', 'pg36452.txt', 'pg36735.txt', 'pg37064.txt', 'pg37160.txt', 'pg37282.txt', 'pg37461.txt', 'pg37523.txt', 'pg37610.txt', 'pg37736.txt', 'pg37806.txt', 'pg37880.txt', 'pg37984.txt', 'pg38143.txt', 'pg38202.txt', 'pg38304.txt', 'pg38401.txt', 'pg38454.txt', 'pg38539.txt', 'pg38622.txt', 'pg38709.txt', 'pg38799.txt', 'pg38892.txt', 'pg38964.txt', 'pg39029.txt', 'pg39127.txt', 'pg39232.txt', 'pg39353.txt', 'pg39435.txt', 'pg39521.txt', 'pg39632.txt', 'pg39700.txt', 'pg39775.txt', 'pg39908.txt', 'pg40009.txt', 'pg40096.txt', 'pg40156.txt', 'pg40370.txt', 'pg40538.txt', 'pg40641.txt', 'pg40769.txt', 'pg40863.txt', 'pg40956.txt', 'pg41055.txt', 'pg41156.txt', 'pg41264.txt', 'pg41343.txt', 'pg41472.txt', 'pg41567.txt', 'pg41685.txt', 'pg41773.txt', 'pg41902.txt', 'pg42048.txt', 'pg42173.txt', 'pg42342.txt', 'pg42473.txt', 'pg42552.txt', 'pg42638.txt', 'pg42736.txt', 'pg42854.txt', 'pg43060.txt', 'pg43254.txt', 'pg43427.txt', 'pg768.txt', 'pg84.txt']
# curl each book
for book in book_list:
  !curl "https://raw.githubusercontent.com/cd-public/books/main/{book}" -o {book}

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  739k  100  739k    0     0  3943k      0 --:--:-- --:--:-- --:--:-- 3955k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  499k  100  499k    0     0  1445k      0 --:--:-- --:--:-- --:--:-- 1443k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1871k  100 1871k    0     0  4942k      0 --:--:-- --:--:-- --:--:-- 4951k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 2036k  100 2036k    0     0  3333k      0 --:--:-- --:--:-- --:--:-- 3332k
  % Total    % Received % Xferd  Average Speed   Tim

In [3]:
def read_txt(filename = 'pg1342.txt'):
  file = open(root + '/' + filename,"r")
  return file.read()
pride = read_txt()

In [7]:
pronouns = ['he','him','his','she','her','hers']
def pronoun_check(x):
  '''
  x = list of tuples
  returns list of tuples with only pronouns
  '''
  pro_toks = []
  male_count, female_count = 0, 0
  for i in x:
    flag = False
    last_pos = ''
    last_tok = ''
    for y in i:
      if flag:
        if pos != 'NN' and pos != 'NNS' and token == '.':
          female_count += 1
          flag = False
          continue
      token, pos = y
      token = token.lower()
      # Magic (identifies instances of 'her' that are objective)
      if token in pronouns:
        if token == 'her':
          flag = True
          last_pos = pos
          last_tok = token
        else:
            if token == 'he' or token == 'him':
              male_count += 1
            else:
              female_count += 1

  return male_count, female_count

#def count_pronouns(x):

In [10]:
def book_cleaner(text):
  text = text.replace('Mr.','Mr')
  text = text.replace('Mrs.','Mrs')
  text = text.replace('Ms.','Ms')
  text = text.replace('Dr.','Dr')
  text = text.replace('_','')
  text = text.replace('-','')
  # text = text.replace('\n', ' ')
  # text = text.replace('""',' ')
  # text = text.replace('  ',' ')
  # # text = text.lower()
  return text

pridesplit = book_cleaner(pride).split('.')

In [11]:
def period(sentences):
  sent_list = []
  for sent in sentences:
    sent = sent+'.'
    sent_list.append(sent)
  return sent_list
pridesplit = period(pridesplit)

for each sentence, find all people mentiond. Add the number of male and female pronounce counted in the sentence to a dict where the key is the persons name.

In [14]:
def book_person_dict_builder(split_book):
  nlp = spacy.load('en_core_web_sm')
  personGenderCount = {} # person: malePronounCount, femalePronounCount
  # for sent in pridesplit:
  for sent in tqdm(split_book): # for each sentence in pride
    doc = nlp(sent) # create the spacy doc
    people = []
    for ent in doc.ents:
      if ent.label_ == 'PERSON':
        people.append(ent.text) # get all people entities
    if len(people) == 0: # if there aren't any people
      continue # move onto next sentence
    annoted_sent = pipeline.annotate(sent) # anotate the sentece with parts of speach
    tok_tag = [(annoted_sent['token'], annoted_sent['pos'])] # list of tuples (token, part of speech)
    zips = [list(zip(tt[0],tt[1])) for tt in tok_tag] # zip
    male_count, female_count = pronoun_check(zips) # get male/female PN counts
    for person in people: # for each person in the sentence
      try: # if they already exist in the the dict
        # add the old counts to the new
        person_male_count = personGenderCount[person][0]
        person_female_count = personGenderCount[person][1]
        personGenderCount[person] = person_male_count + male_count, person_female_count + female_count
      except KeyError: # if they don't exist in the dict, add them
        personGenderCount[person] = male_count, female_count

  return personGenderCount

pride_dict = book_person_dict_builder(pridesplit)


100%|██████████| 5516/5516 [12:55<00:00,  7.12it/s]


{'Project Gutenberg eBook': (0, 0),
 'gutenberg': (0, 0),
 'Jane Austen': (0, 0),
 'Chuck Greif': (0, 0),
 'GEORGE ALLEN\n                               ': (0, 0),
 'George Saintsbury': (0, 0),
 'Hugh Thomson': (0, 1),
 'George Allen': (1, 5),
 'CHANCERY LANE': (0, 0),
 'Comyns Carr': (0, 1),
 'Walt Whitman': (0, 0),
 'Mary': (10, 26),
 'Crawford': (2, 0),
 'Mrs Norris': (2, 0),
 'Emma': (0, 0),
 'Austen': (3, 11),
 'Miss Bates': (0, 0),
 'Miss Austen': (0, 0),
 'Wickham': (64, 86),
 'Mrs Rushworth': (0, 0),
 'Jane': (139, 287),
 'Frank Churchill': (0, 0),
 'Jane Fairfax': (0, 0),
 'Darcy': (276, 431),
 'Elizabeth': (242, 512),
 'John\nThorpe': (0, 0),
 'Swift': (0, 0),
 'Smollett': (0, 0),
 'Miss Burney': (0, 0),
 'Addison': (0, 0),
 'Lydia Bennet': (0, 1),
 'Mrs Musgrove’s': (0, 0),
 'Bennet': (27, 53),
 'Goldwin Smith': (1, 0),
 'Memling': (0, 1),
 'Meissonier': (0, 1),
 'Mary Wollstonecraft': (0, 0),
 'Collins': (52, 78),
 'Mrs\nBennet': (3, 8),
 'Mrs Bennet': (36, 66),
 'Thackeray

convert dict to list so we can itterate through it

In [15]:
def dict_to_list(dict):
  # convert personGenderCount to list
  out = []
  for key, value in dict.items():
    m_count, f_count = value[0], value[1]
    if m_count + f_count < 10:
      continue
    out.append([key, value[0], value[1]])
  return out

personGenderList = dict_to_list(pride_dict)


manually verify each persons gender (do not presume to know unless prefaced by Mr, Mrs, ... etc)

In [19]:
def manual_verify(personGenderList):
  out_list = [] # will consist of [name, prediction, actual]
  old_out_file = pd.read_csv('output.csv')
  for person in personGenderList:
    name, male_count, female_count = person[0], person[1], person[2]
    prediciton = 1 if male_count > female_count else 0
    if name in old_out_file['name'].values:
      actual = old_out_file[old_out_file['name'] == name]['actual'].values[0]
    else:
      actual = input('Is ' + name + ': ' + str(prediciton) + '? ') # 1 for M, 0 for F, -1 for other
    out_list.append([name, prediciton, actual])
  return out_list

out_list = manual_verify(personGenderList)

save to file

In [20]:
def save_books_person_dict(out_list, book):
  df = pd.DataFrame(out_list, columns=['name', 'prediction', 'actual'])
  df = df.astype({'actual': int})
  df.to_csv('book_output_' + datetime.datetime.now().strftime("%Y-%m-%d") + '.csv', index=False)
  return df

df = save_books_person_dict(out_list, 'pride')

In [21]:
df

,name,prediction,actual
0,Mary,0,0
1,Austen,0,0
2,Wickham,0,1
3,Jane,0,0
4,Darcy,0,1
5,Elizabeth,0,0
6,Bennet,0,0
7,Collins,0,1
8,Mrs\nBennet,0,0
9,Mrs Bennet,0,0


calculate accuracy

In [22]:
def accuracy_calc(df, book):
  # calculate accuracy, when predicted == actual that's good
  accuracy = (df['prediction'] == df['actual']).sum() / len(df)
  print(f'Total Accuracy: {round(accuracy*100,2)}%')
  # female accuracy
  female_df = df[df['actual'] == 0]
  female_accuracy = (female_df['prediction'] == female_df['actual']).sum() / len(female_df)
  print(f'Female Accuracy: {round(female_accuracy*100,2)}%')
  # male accuracy
  male_df = df[df['actual'] == 1]
  male_accuracy = (male_df['prediction'] == male_df['actual']).sum() / len(male_df)
  print(f'Male Accuracy: {round(male_accuracy*100,2)}%')

Accuracy: 78.26%


Female Accuracy: 96.67%
Male Accuracy: 43.75%


very good at predicting female characters, terrible at predicting males. This probably indicates the main subjectmatter of this book is women (idk). perhaps we can use this to predict the authors gender.